<a href="https://colab.research.google.com/github/Jarvis-BITS/MIT-midodrine/blob/main/Midodrine_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print("Tensorflow version " + tf.__version__)


Tensorflow version 2.8.0


run if TPU's are *connected*

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

run if GPU's connected

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [9]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')
 

Authenticated


# Demographic analysis


---


**List of variables/column names we should pull**: 

1.   from mimic_core.admissions: subject_id, gender, admittime,	dischtime, deathtime, ethnicity
2.   from derived: weight/height, age, some indicators related to midodrine injection (still in search)




In [10]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [18]:
%%bigquery --project mit-midodrine
SELECT * FROM `physionet-data.mimic_hosp.pharmacy` 
WHERE medication = 'midodrine'
LIMIT 1000

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,...,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,14656374,24236556,90157243,14656374-187,2138-01-16 08:00:00,2138-01-18 06:00:00,midodrine,Unit Dose,Discontinued,2138-01-16 02:40:33,...,None,None,3.0,None,Ongoing,365,Days,None,Omnicell,None
1,13233757,27632812,51036871,13233757-171,2156-08-31 11:00:00,2156-09-08 10:00:00,midodrine,Unit Dose,Discontinued,2156-08-31 10:54:44,...,None,None,3.0,None,Ongoing,365,Days,None,Omnicell,None
2,13944260,20210311,75029595,13944260-74,2161-04-28 18:00:00,2161-05-06 22:00:00,midodrine,Unit Dose,Discontinued via patient discharge,2161-04-28 17:04:09,...,None,None,3.0,None,Ongoing,365,Days,None,Omnicell,None
3,16264276,26629595,50890544,16264276-3135,2112-07-15 09:00:00,2112-07-15 15:00:00,midodrine,Unit Dose,Inactive (Due to a change order),2112-07-15 14:50:55,...,None,None,3.0,None,Ongoing,365,Days,None,LET CALL,None
4,16264276,26629595,51860089,16264276-3156,2112-07-15 08:00:00,2112-07-16 10:00:00,midodrine,Unit Dose,Inactive (Due to a change order),2112-07-15 15:14:15,...,None,None,3.0,None,Ongoing,365,Days,None,LET CALL,None
5,17616449,21469398,39839195,17616449-316,2183-11-16 12:00:00,2183-11-17 13:00:00,midodrine,Unit Dose,Discontinued,2183-11-16 13:45:08,...,None,None,3.0,None,Ongoing,365,Days,None,LET CALL,None
6,13944260,20210311,35019534,13944260-73,2161-04-28 20:00:00,2161-04-28 17:00:00,midodrine,Unit Dose,Inactive (Due to a change order),2161-04-28 15:31:17,...,None,None,3.0,None,Ongoing,365,Days,None,Omnicell,None
7,16264276,26629595,57918012,16264276-3156,2112-07-15 08:00:00,2112-07-18 21:00:00,midodrine,Unit Dose,Discontinued,2112-07-16 10:48:51,...,None,None,3.0,None,Ongoing,365,Days,None,LET CALL,None
8,17616449,21469398,41713510,17616449-316,2183-11-15 21:00:00,2183-11-16 13:00:00,midodrine,Unit Dose,Inactive (Due to a change order),2183-11-15 15:20:06,...,None,None,3.0,None,Ongoing,365,Days,None,LET CALL,None
9,13944260,20210311,74781293,13944260-63,2161-04-28 20:00:00,2161-04-28 15:00:00,midodrine,Unit Dose,Inactive (Due to a change order),2161-04-28 15:19:40,...,None,None,3.0,None,Ongoing,365,Days,None,PATIENT MAY TAKE OWN,None
